In [1]:
import pandas as pd
import os
from CLEAN.infer import *
# pre-compute esm-1b embedding for sequences in new-392 dataset

os.chdir('/disk1/jyang4/repos/CLEAN/app')
ensure_dirs("data/esm_data")
ensure_dirs("data/pretrained")

## CLEAN

In [2]:
dataset = 'price_test_108'
test = pd.read_csv('/disk1/jyang4/repos/ProteinDT_submission/data/PECT/test_sets/{}.csv'.format(dataset))
df = test.rename(columns={'id': 'Entry', 'brenda': 'EC number', 'sequence': 'Sequence'})
df.drop_duplicates(subset='Entry', inplace=True) #for now drop duplicates to make things easier
df = df[['Entry', 'EC number', 'Sequence']]
df.to_csv('data/{}.csv'.format(dataset), index=False, sep='\t')

csv_to_fasta("data/{}.csv".format(dataset), "data/{}.fasta".format(dataset))
retrive_esm1b_embedding(dataset)

Transferred model to GPU
Read data/price_test_108.fasta with 108 sequences
Processing 1 of 11 batches (14 sequences)
Processing 2 of 11 batches (13 sequences)
Processing 3 of 11 batches (12 sequences)
Processing 4 of 11 batches (11 sequences)
Processing 5 of 11 batches (10 sequences)
Processing 6 of 11 batches (10 sequences)
Processing 7 of 11 batches (9 sequences)
Processing 8 of 11 batches (9 sequences)
Processing 9 of 11 batches (8 sequences)
Processing 10 of 11 batches (7 sequences)
Processing 11 of 11 batches (5 sequences)


In [16]:
dataset = "price"

In [3]:
from CLEAN.infer import infer_maxsep
train_data = "split100"
test_data = dataset
infer_maxsep(train_data, test_data, report_metrics=True, pretrained=True)

The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([108, 128])


100%|██████████| 5242/5242 [00:00<00:00, 27272.12it/s]


Calculating eval distance map, between 108 test ids and 5242 train EC cluster centers


108it [00:00, 732.56it/s]


############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 108 | total ec: 39 
>>> precision: 0.573 | recall: 0.426| F1: 0.457 | AUC: 0.713 | accuracy: 0.426 
---------------------------------------------------------------------------
############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 108 | total ec: 39 
>>> precision: 0.573 | recall: 0.426| F1: 0.457 | AUC: 0.713 
---------------------------------------------------------------------------


## diamond BLASTp

In [4]:
swissprot = pd.read_csv('/disk1/jyang4/repos/ProteinDT_submission/data/PECT/CLEANsplit100.csv')
id2ec = ec2smiles = swissprot.groupby('id')['brenda'].apply(list).to_frame().to_dict()['brenda']
id2ec

{'A0A009IHW8': ['3.2.2.6'],
 'A0A023GS28': ['1.14.11.61', '1.14.11.62'],
 'A0A023GS29': ['1.14.11.61', '1.14.11.62'],
 'A0A023W421': ['3.1.1.96'],
 'A0A024BTN9': ['1.4.3.2'],
 'A0A024RBG1': ['3.6.1.52'],
 'A0A024SH20': ['3.2.1.4'],
 'A0A024SMV2': ['1.1.1.179'],
 'A0A059TC02': ['1.2.1.44'],
 'A0A060LAL9': ['1.11.2.3'],
 'A0A060X6Z0': ['1.14.16.2'],
 'A0A061B0Q2': ['2.3.1.268', '3.1.2.1'],
 'A0A061FBW2': ['2.1.1.141'],
 'A0A061FDP1': ['2.1.1.141'],
 'A0A061FKL9': ['2.1.1.159'],
 'A0A061FKM4': ['2.1.1.159'],
 'A0A061FLA2': ['2.1.1.159'],
 'A0A061FMF5': ['2.1.1.159'],
 'A0A061FTC2': ['2.1.1.159'],
 'A0A061I403': ['2.7.7.n1'],
 'A0A067XGX8': ['2.5.1.54'],
 'A0A067YB04': ['2.4.1.170'],
 'A0A067YBQ3': ['2.4.1.170'],
 'A0A068BIF1': ['2.3.1.268'],
 'A0A068J840': ['2.4.1.363'],
 'A0A072UR65': ['3.2.1.14'],
 'A0A072UTP9': ['3.4.22.16'],
 'A0A072VEP0': ['3.2.1.14'],
 'A0A073CEA3': ['4.1.1.100'],
 'A0A075TXZ1': ['4.1.1.52'],
 'A0A077ES70': ['2.1.1.336'],
 'A0A077ESS0': ['2.1.1.336'],
 'A0A077EW86':

In [5]:
dataset = 'price_test_108'
os.chdir('/disk1/jyang4/repos/ProteinDT_submission/data/PECT/test_sets/')
df = pd.read_csv(f'{dataset}.csv')
seqs = df['sequence'].values

In [6]:
#write seqs to fasta format
with open(f'{dataset}.fasta', 'w') as f:
    for i, seq in enumerate(seqs):
        f.write(f'>{i}\n{seq}\n')

In [7]:
!/disk1/jyang4/repos/BLAST/diamond blastp -d /disk1/jyang4/repos/BLAST/swissprot -q price_test_108.fasta -o price_test_108.blastp -k 1

diamond v2.1.9.163 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 384
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: 
#Target sequences to report alignments for: 1
Opening the database...  [0.129s]
Database: /disk1/jyang4/repos/BLAST/swissprot (type: Diamond database, sequences: 570830, letters: 206533160)
Block size = 2000000000
Opening the input file...  [0s]
Opening the output file...  [0s]
Loading query sequences...  [0s]
Masking queries...  [0.029s]
Algorithm: Double-indexed
Building query histograms...  [0.046s]
Seeking in database...  [0s]
Loading reference sequences...  [0.389s]
Masking reference...  [1.192s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.187s]
Allocating buffers...

In [8]:
#blast_results = pd.read_csv('testA_3668.blastp', sep='\t', header=None)
#blast_results = pd.read_csv('testB_1591.blastp', sep='\t', header=None)
blast_results = pd.read_csv('price_test_108.blastp', sep='\t', header=None)
blast_results

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,sp|F2JVT6|MANI_MARM1,51.4,399,181,4,6,403,3,389,2.350000e-146,424.0
1,1,sp|Q1JUQ0|KDADA_AZOBR,80.5,293,57,0,1,293,16,308,2.110000e-169,474.0
2,2,sp|Q9A9Z0|XDH_CAUVC,47.4,249,124,4,31,279,6,247,1.930000e-67,213.0
3,3,sp|Q9GKX6|GALM_PIG,39.2,319,184,4,27,344,26,335,2.610000e-70,226.0
4,4,sp|P0DOV6|SGL_PSEPU,36.5,282,172,5,19,298,15,291,4.600000e-49,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,102,sp|Q888H1|URODH_PSESM,82.8,274,47,0,1,274,1,274,8.870000e-167,465.0
83,104,sp|A0QXD8|ELTD_MYCS2,50.7,355,171,3,6,357,8,361,2.290000e-129,377.0
84,105,sp|A0A0P9JFY5|MANI_PSEC1,69.7,402,122,0,9,410,9,410,1.170000e-205,575.0
85,106,sp|P78061|PUUA_ECOLI,43.4,445,250,2,12,455,26,469,2.310000e-125,375.0


In [9]:
df['index'] = df.index
merged = pd.merge(df, blast_results, left_on='index', right_on=0, how = 'left')
merged

merged['predicted_ecs'] = merged[1].str.split('|').str[-2].map(id2ec)
merged.dropna(subset=['predicted_ecs'], inplace=True) #for now drop the ones where the mapping failed

correct = 0
#loop through each row of merged and check if brenda is in predicted_ecs
for index, brenda in zip(merged.index, merged['brenda'].values):
    if brenda in merged.loc[index, 'predicted_ecs']:
        correct += 1
accuracy = correct/len(merged)
print(accuracy)

#accuracy = (merged['brenda'].isin(merged['predicted_ecs'])).mean()
#print(accuracy)

0.45
